# Building Julia Apps and So Can You! <br><small>Compiling and distributing desktop applications written in Julia</small>

### Nathan Daly <br><small>nhdaly@gmail.com<br>http://nhdaly.github.io</small><br><br>
    

Download this notebook here:<br>
https://github.com/NHDaly/juliaCon2018-ApplicationBuilder-Slides

## This Notebook

Most of this presentation is in the form of an interactive jupyter notebook. If you've downloaded it ([here](https://github.com/NHDaly/juliaCon2018-ApplicationBuilder-Slides)), you can open the notebook like this:
```julia
julia> using IJulia; IJulia.notebook()
```
Then double click the file you downloaded.

## Overview

- Definitions
- Demo
- Building a simple command line Application
- Building an Application with a GUI (dealing with binary dependencies)

In [ ]:
# NOTE: Below are the packages needed to run all of this notebook.
# You really need ApplicationBuilder. The rest are optional.
Pkg.clone("https://github.com/NHDaly/ApplicationBuilder.jl")  # Necessary for whole notebook
Pkg.add("PrintFileTree")  # Not necessary (you can use unix `find` as a decent replacement).
Pkg.add("UnicodePlots")  # Only needed for the demo application.
Pkg.add("Plots");  # Only needed for the GUI example at the end.
Pkg.add("Blink"); using Blink; Blink.AtomShell.install()  # Only needed if you want to run the GUI example at the end.

On Julia v0.7:
```julia
pkg> add https://github.com/NHDaly/ApplicationBuilder.jl.git#julia0.7
pkg> add https://github.com/NHDaly/ApplicationBuilderRuntimeUtils.jl.git
```

What is an Application?

Julia 0.7 [Pkg docs](https://docs.julialang.org/en/latest/stdlib/Pkg/#Glossary-1):

> <u>Application:</u> a project which provides standalone functionality not intended to be reused by other Julia projects.

What am I talking about when I say "Application"?

- Provides standalone functionality (same as Julia 0.7 definition)

- Distributable

- Self-contained (no Julia installation required)

- Compiled binary

### Demo: Paddle Battle

![Paddle Battle Clip](https://nhdaly.github.io/assets/images/Paddle-Battle-Clip.gif)

An example Application: Paddle Battle is a simple Pong-style game written entirely in Julia using a C graphics library called SDL.
https://github.com/NHDaly/PaddleBattleJL<br>
[Paddle Battle on Mac App Store](https://itunes.apple.com/us/app/paddle-battle/id1370991412?mt=12)

Julia is great for building Applications.

<small>First of all, it's fun! Look at me live-editing code in Juno:</small>
![Live editing collision handling](https://nhdaly.github.io//assets/images/Paddle-Battle-Juno-live-editing.gif)



> That's nice... But why do I want to compile my julia program into an Application?

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; To share your work!

## Making our own App

In [ ]:
mkpath("OurProject")

In [ ]:
mkpath("OurProject/src")  # For OurProject's source code

In [ ]:
write("OurProject/src/project.jl",
 """
    using UnicodePlots

    println("**** Hello From Julia! ****")
    r = rand(0:2π)
    println(lineplot(1:100, sin.(linspace(r, r+2π, 100))))
    
    println("Current working directory:", pwd())
 """
)

In [ ]:
using PrintFileTree
printfiletree("OurProject")

In [ ]:
include("OurProject/src/project.jl")

## ~~ ApplicationBuilder.jl ~~

In [ ]:
 using ApplicationBuilder; using BuildApp  # BuildApp was installed automatically, "next to" ApplicationBuilder.

### What does “compiling” mean? What's it doing?

- [PackageCompiler.jl](https://github.com/JuliaLang/PackageCompiler.jl) compiles your julia program to a binary.
  - `julia -Cx86-64 --compile=yes`<br>`--depwarn=yes --precompiled=no`<br>`--compilecache=no --output-o`...
- Remember: Julia is a compiled language with an interpreter, not the other way around.


This creates an "object file", which contains all the compiled functions from your code plus all the rest of the julia standard library. 

And finally, `PackageCompiler` then links that into a shared library, and ultimately creates an executable.

So much work was done here by:
<div class="row" style="display: flex; padding: 20px;">
  <div class="column" style="float: left; width: 35%;">

- SimonDanisch
- lucatrv
- vtjnash
  </div><div class="column" style="float: left; width: 65%;">


- timholy
- ViralBShah
- ... and [many more](https://github.com/JuliaLang/PackageCompiler.jl/graphs/contributors)

</div>
</div>

THANKS!

In [ ]:
build_app_bundle("OurProject/src/project.jl")

 *What's going on there?*
 - that's really slow!
 - Our `println` was executed!
 - It **FAILED!?** Something about undefined symbols...?
    - More on this below:

## What happens when you build an "app bundle?"
Quick aside: pulling back the curtain...

### 1. ApplicationBuilder creates an "Application Bundle" based on the supplied configuration.

"Application Bundle" is my term for a standard, OS-native application.

These "bundles" wrap up all the things necessary to run the application:
 - an executable,
 - supporting runtime libraries, and
 - supporting resources (graphics, fonts, sounds, etc.)
 
On Mac, this is called an "app", and it's actually just a directory with the `.app` extension.

On Windows and Linux, just a standard directory.

### 2. ApplicationBuilder turns your code into an executable.

The executable is made out of two pieces:
 1. Build a static shared library (a sysimg) containing your parsed and compiled code.
 2. Compile a tiny "driver" executable which initializes the julia runtime, loads your code, and runs it.
 
The entry point that the *driver executable* uses for your code is a function called `julia_main()`. This maps to the the way `C` programs are invoked.

**What this means for you** is that your julia program *must* contain a `julia_main` function, which will be the first thing called when your application is run.

To see an example of `julia_main`, let's take a look at some of the examples.

`ApplicationBuilder.jl/examples/`:
&nbsp;&nbsp;&nbsp;&nbsp;https://github.com/NHDaly/ApplicationBuilder.jl/tree/master/examples

In [ ]:
write("OurProject/src/project.jl",
 """
    using UnicodePlots
    
    println("**** Hello from the outside! ****")
    
    Base.@ccallable function julia_main(ARGS::Vector{String})::Cint
        println("**** Hello From Julia! ****")
        r = rand(0:2π)
        println(lineplot(1:100, sin.(linspace(r, r+2π, 100))))
    
        println("Current working directory:", pwd())

        return 0
    end
 """
)

In [ ]:
build_app_bundle("OurProject/src/project.jl",
                 appname="HelloWorld",
                 builddir="OurProject/builds",
                 commandline_app=true)

Explanation of `julia_main`'s (ugly) function declaration:
```julia
Base.@ccallable
function julia_main(ARGS::Vector{String})::Cint
```
 - `Base.@ccallable` allows the function to be called by the *driver* executable.
 - `::Cint` is the status code returned from your program (the same as in C/C++).

Let's run the application we just built!

In [ ]:
run(`open ./OurProject`)

One more note on compiling: Snoopfiles!

In order to see a _speed up_ from compiling, you have to _fully compile_ your code to native instructions, not just lower it to an AST. `snoop` == tell julia what to compile.

```julia
build_app_bundle(
    "OurProject/src/project.jl",
    autosnoop=true,  # <----- Added this yesterday...
  # or:
    snoopfile="project_snoopfile.jl",
)
```

## Ship it!!

And that's it! `HelloWorld.app` is a real, complete application that can be distributed to real users, either by downloading from a website, or getting it from an App Store.

But can it gui!?

# Adding a GUI to our App

## Julia desktop GUI Packages
And my success using them with ApplicationBuilder.
  
<div style="display: flex; padding: 20px;">
  <div style="float: left; width: 35%;">
  
-  ✅ [Blink.jl](https://github.com/JunoLab/Blink.jl) 
-  ❌ [Gtk.jl](https://github.com/JuliaGraphics/Gtk.jl)  
-  ✅ [Libui.jl](https://github.com/joa-quim/Libui.jl)
-   **?**  &nbsp; [Cairo.jl](https://github.com/JuliaGraphics/Cairo.jl)
 </div><div style="float: left; width: 65%;">


-  ✅ [SimpleDirectMediaLayer.jl](https://github.com/jonathanBieler/SimpleDirectMediaLayer.jl)
-  ❌ [Tk.jl](https://github.com/JuliaGraphics/Tk.jl)
-  ❌ [QML.jl](https://github.com/barche/QML.jl) 
-   **?**  &nbsp; [Win32GUIDemo.jl](https://github.com/ihnorton/Win32GUIDemo.jl)

</div>
</div>


I'm still working on support for the ones that currently aren't working. Come talk to me if you're interested!

In [ ]:
using Blink

win = Window(); sleep(2)

In [ ]:
body!(win, """
    <input id="mySlider" type="range" min="1" max="100" value="50">
    <script> 
        var mySlider = document.getElementById("mySlider")
    </script>
"""); sleep(2)
tools(win)

Blink.@js_ win console.log("HELLO!")
Blink.@js_ win mySlider.oninput = 
    (e) -> (Blink.msg("sliderChange", mySlider.value);
            console.log("sent msg to julia!"); e.returnValue=false)
Blink.handlers(win)["sliderChange"] = (val) -> (println("msg from js: $val"))

In [ ]:
using Plots
plotly()

In [ ]:
r = rand(0:0.1:2π)
p = plot(r:2π/100:r+2π, sin)

In [ ]:
# Let's draw that plot as an SVG and show it in our html window!
buf = IOBuffer()
show(buf, MIME("text/html"), p)
plothtml = String(take!(buf))

body!(win, """<script>var Plotly = require('$(Plots._plotly_js_path)');</script>
              <div id="plotHolder"></div>"""); sleep(3)
content!(win, "#plotHolder", plothtml);

### Putting it together!

In [ ]:
using Blink, Plots
plotly()

win = Window(); sleep(2)
body!(win, """
    <input id="mySlider" type="range" min="1" max="100" value="50">
    <div id="plotHolder">
        plot goes here...
    </div>
    <script>
        mySlider = document.getElementById("mySlider")
        var Plotly = require('$(Plots._plotly_js_path)');
    </script>
"""); sleep(2)
tools(win)

Blink.@js win console.log("HELLO!")
Blink.@js win mySlider.oninput = 
    (e) -> (Blink.msg("sliderChange", mySlider.value);
            console.log("sent msg to julia!"); e.returnValue=false)

function sliderChange(val)
    r = parse(val)
    p = Plots.plot(r:2π/100:r+2π, sin)
    buf = IOBuffer()
    show(buf, MIME("text/html"), p)
    plothtml = String(take!(buf))

    content!(win, "#plotHolder", plothtml, fade=false)
end

Blink.handlers(win)["sliderChange"] = sliderChange

Note that it still is pretty slow.

I think this is mainly because we haven't statically-compiled anything (we've only lowered our code) because we're not using any snoop-compiling. However, `autosnoop` fails for Blink programs right now.

## Building a static Application!

In [ ]:
write("OurProject/src/project.jl",
 raw"""
    using Blink, Plots
    plotly()

    Base.@ccallable function julia_main(ARGS::Vector{String})::Cint
        win = Blink.Window(); sleep(2)
        Blink.body!(win, \"\"\"
            <input id="mySlider" type="range" min="1" max="100" value="50">
            <div id="plotHolder">
                plot goes here...
            </div>
            <script>
                mySlider = document.getElementById("mySlider")
                var Plotly = require('$(Plots._plotly_js_path)');
            </script>
        \"\"\"); sleep(2)
        Blink.tools(win)

        Blink.@js_ win console.log("HELLO!")
        Blink.@js_ win mySlider.oninput = 
            (e) -> (Blink.msg("sliderChange", mySlider.value);
                    console.log("sent msg to julia!"); e.returnValue=false)

        function sliderChange(val)
            r = parse(val)
            p = Plots.plot(r:2π/100:r+2π, sin);  # Don't forget this ';' to prevent it opening a plot window!
            buf = IOBuffer()
            # invokelatest b/c show compiles more functions, and fails due to world age (https://discourse.julialang.org/t/running-in-world-age-x-while-current-world-is-y-errors/5871/5)
            Base.invokelatest(show, buf, MIME("text/html"), p);
            plothtml = String(take!(buf))

            Blink.content!(win, "#plotHolder", plothtml, fade=false)
        end

        Blink.handlers(win)["sliderChange"] = sliderChange
    
        # Keep the process alive until the window is closed!
        while Blink.active(win)
            sleep(1)
        end

        return 0
    end
 """
)

In [ ]:
build_app_bundle(
    "OurProject/src/project.jl",
    appname="SinePlotter",  # New App name
    builddir="OurProject/builds",
)

In [ ]:
run(`open OurProject/builds/SinePlotter.app`)

But how do we make a _distributable_ Blink app?


We need to wrap up all the things Blink needs to run.

Example:

https://github.com/NHDaly/ApplicationBuilder.jl/blob/master/examples/blink.jl

In [ ]:
# Apply that to our program, and this is what we have:
write("OurProject/src/project.jl",
 raw"""
    using Blink, Plots
    using ApplicationBuilder

    # THIS IS NEEDED FOR YOUR CODE TO RUN ON ANY COMPUTER
    if get(ENV, "COMPILING_APPLE_BUNDLE", "false") == "true"
        println("Overriding Blink dependency paths.")
        eval(Blink.AtomShell, :(_electron = "Julia.app/Contents/MacOS/Julia"))
        eval(Blink.AtomShell, :(mainjs = "main.js"))
        eval(Blink, :(buzz = "main.html"))
        eval(Blink, :(resources = Dict("spinner.css" => "res/spinner.css",
                                 "blink.js" => "res/blink.js",
                                 "blink.css" => "res/blink.css",
                                 "reset.css" => "res/reset.css")))
        # Clear out Blink.__inits__, since it will attempt to evaluate hardcoded paths.
        # (We've defined all the variables manually, above: `resources` and `port`.)
        eval(Blink, :(empty!(__inits__)))

        eval(HttpParser, :(lib = basename(lib)))
        eval(MbedTLS, :(const libmbedcrypto = basename(libmbedcrypto)))

        using WebSockets
        eval(WebSockets, :(using HttpServer))  # needed to cause @require lines to execute & compile
        eval(WebSockets,
            :(include(joinpath(Pkg.dir("WebSockets"),"src/HttpServer.jl"))))  # Manually load this from the @requires line.

        println("Done changing Blink dependencies.")
        
        println("Overriding Plotly dependency paths.")
        eval(Plots, :(_plotly_js_path = "plotly-latest.min.js"))
        println("Done changing Plotly dependencies.")
    end
    
    Base.@ccallable function julia_main(ARGS::Vector{String})::Cint
        # THIS IS NEEDED FOR YOUR CODE TO RUN ON ANY COMPUTER
        ApplicationBuilder.App.change_dir_if_bundle()

        # This must be inside app_main() b/c must be after `change_dir_if_bundle()`
        Plots.plotly()

        # Set Blink port randomly before anything else, so it's not compiled with a fixed port.
        eval(Blink, :(const port = get(ENV, "BLINK_PORT", rand(2_000:10_000))))

        win = Blink.Window(); sleep(5)
        Blink.body!(win, \"\"\"
            <input id="mySlider" type="range" min="1" max="100" value="50">
            <div id="plotHolder">
                plot goes here...
            </div>
            <script>
                mySlider = document.getElementById("mySlider")
                var Plotly = require('../../../../../$(Plots._plotly_js_path)');
            </script>
        \"\"\"); sleep(2)
        Blink.tools(win)

        Blink.@js_ win console.log("HELLO!")
        Blink.@js_ win mySlider.oninput = 
            (e) -> (Blink.msg("sliderChange", mySlider.value);
                    console.log("sent msg to julia!"); e.returnValue=false)

        function sliderChange(val)
            r = parse(val)
            p = Plots.plot(r:2π/100:r+2π, sin);  # Don't forget this ';' to prevent it opening a plot window!
            buf = IOBuffer()
            # invokelatest b/c show compiles more functions, and fails due to world age (https://discourse.julialang.org/t/running-in-world-age-x-while-current-world-is-y-errors/5871/5)
            Base.invokelatest(show, buf, MIME("text/html"), p);
            plothtml = String(take!(buf))

            Blink.content!(win, "#plotHolder", plothtml, fade=false)
        end

        Blink.handlers(win)["sliderChange"] = sliderChange
    
        # Keep the process alive until the window is closed!
        while Blink.active(win)
            sleep(1)
        end

        return 0
    end
"""
)

In [ ]:
# Build a distributable SinPlotter.app!
using ApplicationBuilder; using BuildApp
using Blink, Plots
blinkPkg = Pkg.dir("Blink")
macroToolsPkg = Pkg.dir("MacroTools")

build_app_bundle(
    "OurProject/src/project.jl",
    appname="SinePlotterBundled",
    builddir="OurProject/builds",
    resources = [
        # Blink resources
        joinpath(blinkPkg, "deps","Julia.app"),
        Blink.AtomShell.mainjs,
        joinpath(blinkPkg, "src","content","main.html"),
        joinpath(blinkPkg, "res"),
        # Plots resources
        Plots._plotly_js_path,
    ],
    libraries = [
        HttpParser.lib,
        MbedTLS.libmbedcrypto,
    ],
)

In [ ]:
run(`open OurProject/builds/`)

## Future Work <small>... with you!?...</small>

- Bloat: Do we really need all of those julia libraries? (100M!) Does _pong_ need FFT?
    - Can we maybe even cut unused code out of the sysimg? (40M!)
- \*Binary dependencies\*:   (the biggest pain point right now.)
    - Automate detection of binary dependencies?
    - Automate copying and bundling?
    - ✨ Automate modifying hard-coded paths? ✨
        - **Can we have Packages handle this!?**
        - BinDeps, BinaryProvider, BinDeps2, BinaryBuilder, etc: can we make them aware of the compilation process?

## Special Thanks

- ranjanan
- lucatrv
- simondanish
- vtjnash